In [15]:

import json
import torch
import logging
import numpy as np 
import pandas as pd 
import evaluate
from evaluate import load


In [2]:
from transformers import T5Tokenizer,T5ForConditionalGeneration
import json

c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    logging.info(f'Using GPU: {torch.cuda.get_device_name()}')
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    logging.info('Using CPU')
    print('Using CPU')

Using GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [ ]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

In [4]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small").to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
file_path = "data/Task2/"

In [6]:
testing_data = []
validation_data = []

In [91]:
# load json
with open(file_path + "test.json") as f:
    testing_data = json.load(f)
    testing_data = testing_data["translation"]

with open(file_path + "validation_data.json") as f:
    validation_data = json.load(f)
    validation_data = validation_data["translation"]


en_test_data = []
en_val_data = []

de_test_data = []
de_val_data = []

for i in range(len(testing_data)):
    en_test_data.append(testing_data[i]["en"])
    de_test_data.append(testing_data[i]["de"])
for i in range(len(validation_data)):
    de_val_data.append(validation_data[i]["de"])
    en_val_data.append(validation_data[i]["en"])




In [93]:

# en_test_data=en_test_data[:100]
# de_test_data=de_test_data[:100]
# en_val_data=en_val_data[:100]
# de_val_data=de_val_data[:100]



In [94]:
len(en_test_data),len(de_test_data),len(en_val_data),len(de_val_data)

(2998, 2998, 2168, 2168)

In [13]:
prefix = "translate English to German: "
test_model_input=[]
val_model_input=[]
for i in range(len(en_test_data)):
    test_model_input.append(prefix + en_test_data[i])
for i in range(len(en_val_data)):
    val_model_input.append(prefix + en_val_data[i])



In [72]:
test_output = []

j=0;
for i in test_model_input:
    input_ids = tokenizer(i, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, max_length=256, num_beams=5, early_stopping=True)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    test_output.append(decoded_output)
    j+=1
    if( j%10 == 0):
        print(j)

len(test_output)


100


100

In [73]:
# i=50
# print(en_test_data[i],de_test_data[i],test_output[i],sep="\n")
# input_ids = tokenizer(test_model_input[i], return_tensors="pt").input_ids.to(device)
# output = model.generate(input_ids, max_length=256)
# decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
# decoded_output

{'bleu': 0.41678909373773215, 'precisions': [0.6959890610756609, 0.47659980897803245, 0.35456369107321967, 0.26610348468849], 'brevity_penalty': 0.9909256523892572, 'length_ratio': 0.99096657633243, 'translation_length': 2194, 'reference_length': 2214}


In [ ]:
results = bleu.compute(predictions=test_output, references=de_test_data)
print(results)

In [81]:
# meteor
results = meteor.compute(predictions=test_output, references=de_test_data)
print(results)

{'meteor': 0.6658749178164521}


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [83]:

results = bertscore.compute(predictions=test_output, references=de_test_data, lang="de")


tokenizer_config.json: 100%|██████████| 49.0/49.0 [00:00<?, ?B/s]
c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\acer\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config.json: 100%|

In [87]:
print(np.average(results["f1"]))
print(np.average(results["precision"]))
print(np.average(results["recall"]))

0.8949023234844208
0.8967794805765152
0.8932868671417237


In [ ]:
val_output = []

j=0;
for i in val_model_input:
    input_ids = tokenizer(i, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, max_length=256, num_beams=5, early_stopping=True)
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    val_output.append(decoded_output)
    j+=1
    if( j%10 == 0):
        print(j)

len(val_output)

In [ ]:

results = bleu.compute(predictions=val_output, references=de_val_data)
print(results)


In [ ]:
results = meteor.compute(predictions=val_output, references=de_val_data)
print(results)



In [ ]:
results = bertscore.compute(predictions=val_output, references=de_val_data, lang="de")
print(np.average(results["f1"]))
print(np.average(results["precision"]))
print(np.average(results["recall"]))
